In [ ]:
import pandas as pd
#import psycopg2
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# # Connection parameters, yours will be different
# param_dic = {
#     "host"      : "localhost",
#     "database"  : "mqtt",
#     "user"      : "postgres",
#     "password"  : ""
# }
# def connect(params_dic):
#     """ Connect to the PostgreSQL database server """
#     conn = None
#     try:
#         # connect to the PostgreSQL server
#         print('Connecting to the PostgreSQL database...')
#         conn = psycopg2.connect(**params_dic)
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         sys.exit(1) 
#     print("Connection successful")
#     return conn

In [ ]:
# def postgresql_to_dataframe(conn, select_query):
#     """
#     Tranform a SELECT query into a pandas dataframe
#     """
#     cursor = conn.cursor()
#     try:
#         cursor.execute(select_query)
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         cursor.close()
#         return 1
    
#     num_fields = len(cursor.description)
#     field_names = [i[0] for i in cursor.description]
    
#     # Naturally we get a list of tupples
#     tupples = cursor.fetchall()
#     cursor.close()
    
#     # We just need to turn it into a pandas dataframe
#     df = pd.DataFrame(tupples, columns=field_names)
# #     df = pd.DataFrame(tupples)
#     return df

In [ ]:
# # Connect to the database
# conn = connect(param_dic)

In [ ]:
# df_cnc3 = postgresql_to_dataframe(conn, "select * from cnc_3_2")

if run on local machine - then reading from postgres db can be used

instead - reading from csv file

In [ ]:
df_cnc3 = pd.read_csv('cnc_3_2_202312191128.csv')

In [ ]:
df_cnc3.info()

In [ ]:
df_cnc3.isna().sum()

In [ ]:
df_cnc3.head()

In [ ]:
df_cnc3['c1_stat'].unique()

In [ ]:
df_cnc3['c1_mode'].unique()

In [ ]:
df_cnc3['c1_prog'].unique()

# 1.2

In [ ]:
df_cnc3['time_load'] = pd.to_datetime(df_cnc3['time_load'])

In [ ]:
df_cnc3['time_load_msc'] = df_cnc3['time_load'] + pd.Timedelta(hours=3)

In [ ]:
df_cnc3=df_cnc3.sort_values(by=['time_load_msc'])

In [ ]:
df_cnc3['core_work_time_ttl_value'] = pd.to_timedelta(df_cnc3['core_work_time_ttl_value'])

In [ ]:
df_cnc3['core_work_time_cur_value'] = pd.to_timedelta(df_cnc3['core_work_time_cur_value'])

In [ ]:
df_cnc3['c1_tot_work_time'] = pd.to_timedelta(df_cnc3['c1_tot_work_time'])

In [ ]:
df_cnc3['c1_prog_work_time'] = pd.to_timedelta(df_cnc3['c1_prog_work_time'])

In [ ]:
# Plotting the time series chart with Plotly Express
fig = px.line(df_cnc3, x='time_load_msc', y='c1_tot_work_time', 
              labels={'c1_tot_work_time':'Channel Total Work Time Value', 'time_load_msc':'Time'},
              title='Channel Total Work Time')

# Show the plot
fig.show()

In [ ]:
# Plotting the time series chart with Plotly Express
fig = px.line(df_cnc3, x='time_load_msc', y='core_work_time_cur_value', 
              labels={'core_work_time_cur_value':'Core Work Time Current Value', 'time_load_msc':'Time'},
              title='Core Work Time Current')

# Show the plot
fig.show()

In [ ]:
df_cnc3['time_load_msc_next'] = df_cnc3['time_load_msc'].shift(-1)

In [ ]:
df_cnc3['duration'] = df_cnc3['time_load_msc_next']-df_cnc3['time_load_msc']

In [ ]:
df_cnc3.groupby(by=['c1_stat'], as_index=False) \
    .agg({'duration':'sum'})

In [ ]:
df_cnc3['duration_percentage'] = df_cnc3['duration'] / df_cnc3['duration'].sum() * 100

In [ ]:
df_cnc3_agg_stat = df_cnc3.groupby(by=['c1_stat'], as_index=False) \
    .agg({'duration':'sum'})

In [ ]:
df_cnc3_agg_stat['duration_ttl'] = df_cnc3_agg_stat['duration'].sum()

In [ ]:
df_cnc3_agg_stat['duration_percent'] = df_cnc3_agg_stat['duration']/df_cnc3_agg_stat['duration_ttl']*100

In [ ]:
fig = px.bar(df_cnc3_agg_stat, x='c1_stat', y='duration_percent',
             labels={'duration': 'Duration', 'c1_stat': 'c1_stat'},
             title='Stacked Bar Chart by c1_stat', barmode = 'stack')

# Show the plot
fig.show()

In [ ]:
# Plotting the scatter plot with states over time
fig = px.scatter(df_cnc3, x='time_load_msc', y='c1_stat',
                 labels={'c1_stat': 'Status', 'time_load_msc': 'Time'},
                 title='Status Over Time')

# Show the plot
fig.show()

In [ ]:
# Plotting the scatter plot with states over time
fig = px.scatter(df_cnc3, x='time_load_msc', y='c1_mode',
                 labels={'c1_mode': 'Mode', 'time_load_msc': 'Time'},
                 title='Mode Over Time')

# Show the plot
fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    # Plotting the scatter plot with states over time
    fig = px.line(df_cnc3[df_cnc3['c1_prog']==p], x='time_load_msc', y='c1_percent_prog',
                     labels={'c1_percent_prog': 'Percent programm', 'time_load_msc': 'Time'},
                     title=f'Status Over Time: {p}')

    # Show the plot
    fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    # Plotting the scatter plot with states over time
    fig = px.line(df_cnc3[df_cnc3['c1_prog']==p], x='time_load_msc', y='c1_prog_work_time',
                     labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'Status Over Time: {p}')

    # Show the plot
    fig.show()

In [ ]:
df_cnc3['c1_prog'].unique()

In [ ]:
df_cnc3.columns

## channel 1 coordinates

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    fig = go.Figure(data=[go.Scatter3d(
        x=df_cnc3[df_cnc3['c1_prog']==p]['axes_c1_x_float'],
        y=df_cnc3[df_cnc3['c1_prog']==p]['axes_c1_y_float'],
        z=df_cnc3[df_cnc3['c1_prog']==p]['axes_c1_z_float'],
        mode='markers',
        marker=dict(size=5, color='blue'),
    )])
    
    fig.update_layout(scene=dict(xaxis_title='X Coordinate', yaxis_title='Y Coordinate', zaxis_title='Z Coordinate'),
                  title=f'XYZ Coordinates: {p}')

    fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/10_02.nc":
        fig = go.Figure(data=[go.Scatter3d(
            x=df_cnc3[df_cnc3['c1_prog']==p]['axes_c1_x_float'],
            y=df_cnc3[df_cnc3['c1_prog']==p]['axes_c1_y_float'],
            z=df_cnc3[df_cnc3['c1_prog']==p]['axes_c1_z_float'],
            mode='markers',
            marker=dict(size=5, color='blue'),
        )])
        
        fig.update_layout(scene=dict(xaxis_title='X Coordinate', yaxis_title='Y Coordinate', zaxis_title='Z Coordinate'),
                      title=f'XYZ Coordinates: {p}')

        fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_04.nc":
    # Plotting the scatter plot with states over time
        fig = px.scatter(df_cnc3[df_cnc3['c1_prog']==p], x='axes_c1_b_float', y='axes_c1_c_float',
#                      labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'B C coordinates: {p}')

    # Show the plot
        fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_04.nc":
    # Plotting the scatter plot with states over time
        fig = px.scatter(df_cnc3[df_cnc3['c1_prog']==p], x='axes_c1_s_float', y='axes_c1_s2_float',
#                      labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'S S2 coordinates: {p}')

    # Show the plot
        fig.show()

## channel 2 coordinates

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_04.nc":
        fig = go.Figure(data=[go.Scatter3d(
        x=df_cnc3[df_cnc3['c1_prog']==p]['axes_c2_x_float'],
        y=df_cnc3[df_cnc3['c1_prog']==p]['axes_c2_y_float'],
        z=df_cnc3[df_cnc3['c1_prog']==p]['axes_c2_z_float'],
        mode='markers',
        marker=dict(size=5, color='blue'),
    )])
    
        fig.update_layout(scene=dict(xaxis_title='X Coordinate', yaxis_title='Y Coordinate', zaxis_title='Z Coordinate'),
                  title=f'XYZ Coordinates: {p}')

        fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
     if p == "/NC/examples/05_04.nc":
    # Plotting the scatter plot with states over time
        fig = px.scatter(df_cnc3[df_cnc3['c1_prog']==p], x='axes_c2_b_float', y='axes_c2_c_float',
#                      labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'B C coordinates: {p}')

    # Show the plot
        fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_04.nc":
    # Plotting the scatter plot with states over time
        fig = px.scatter(df_cnc3[df_cnc3['c1_prog']==p], x='axes_c2_s_float', y='axes_c2_s2_float',
#                      labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'S S2 coordinates: {p}')

    # Show the plot
        fig.show()

## channel 3 coordinates

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_17.nc":
        fig = go.Figure(data=[go.Scatter3d(
        x=df_cnc3[df_cnc3['c1_prog']==p]['axes_c3_x_float'],
        y=df_cnc3[df_cnc3['c1_prog']==p]['axes_c3_y_float'],
        z=df_cnc3[df_cnc3['c1_prog']==p]['axes_c3_z_float'],
        mode='markers',
        marker=dict(size=5, color='blue'),
    )])
    
        fig.update_layout(scene=dict(xaxis_title='X Coordinate', yaxis_title='Y Coordinate', zaxis_title='Z Coordinate'),
                  title=f'XYZ Coordinates: {p}')

        fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_04.nc":
    # Plotting the scatter plot with states over time
        fig = px.scatter(df_cnc3[df_cnc3['c1_prog']==p], x='axes_c3_b_float', y='axes_c3_c_float',
#                      labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'B C coordinates: {p}')

    # Show the plot
        fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_04.nc":
    # Plotting the scatter plot with states over time
        fig = px.scatter(df_cnc3[df_cnc3['c1_prog']==p], x='axes_c3_s_float', y='axes_c3_s2_float',
#                      labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'S S2 coordinates: {p}')

    # Show the plot
        fig.show()

## channel 4 coordinates

In [ ]:
for p in df_cnc3['c1_prog'].unique():
      if p == "/NC/examples/05_04.nc":
        fig = go.Figure(data=[go.Scatter3d(
            x=df_cnc3[df_cnc3['c1_prog']==p]['axes_c4_x_float'],
            y=df_cnc3[df_cnc3['c1_prog']==p]['axes_c4_y_float'],
            z=df_cnc3[df_cnc3['c1_prog']==p]['axes_c4_z_float'],
            mode='markers',
            marker=dict(size=5, color='blue'),
    )])
    
        fig.update_layout(scene=dict(xaxis_title='X Coordinate', yaxis_title='Y Coordinate', zaxis_title='Z Coordinate'),
                  title=f'XYZ Coordinates: {p}')

        fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_17.nc":
    # Plotting the scatter plot with states over time
        fig = px.scatter(df_cnc3[df_cnc3['c1_prog']==p], x='axes_c4_b_float', y='axes_c4_c_float',
#                      labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'B C coordinates: {p}')

    # Show the plot
        fig.show()

In [ ]:
for p in df_cnc3['c1_prog'].unique():
    if p == "/NC/examples/05_17.nc":
    # Plotting the scatter plot with states over time
        fig = px.scatter(df_cnc3[df_cnc3['c1_prog']==p], x='axes_c4_s_float', y='axes_c4_s2_float',
#                      labels={'c1_prog_work_time': 'Program work time', 'time_load_msc': 'Time'},
                     title=f'S S2 coordinates: {p}')

    # Show the plot
        fig.show()

# 3.1

In [ ]:
df_cnc3 = df_cnc3.sort_values(by=['time_load'])

In [ ]:
df_cnc3.columns

In [ ]:
# df_cnc3['time_load_next'] = df_cnc3['time_load'].shift(-1)

In [ ]:
# df_cnc3['time_load'] = pd.to_datetime(df_cnc3['time_load'])
# df_cnc3['time_load_next'] = pd.to_datetime(df_cnc3['time_load_next'])
# df_cnc3['duration'] = df_cnc3['time_load_next']-df_cnc3['time_load']

In [ ]:
df_cnc3_stat_agg = df_cnc3.groupby(by=['c1_stat'], as_index=False) \
    .agg({'channel_cnt':'count','duration':'sum'}) \
    .rename(columns={'channel_cnt':'cnt'})

In [ ]:
df_cnc3_stat_agg

In [ ]:
df_cnc3_stat_agg['percentage'] = (df_cnc3_stat_agg['duration'] / df_cnc3_stat_agg['duration'].sum()) * 100

In [ ]:
fig = px.pie(df_cnc3_stat_agg, values='percentage', names='c1_stat', 
             title='Channel 1 states percentage',
             hover_data=['percentage'], labels={'percentage':'Percentage'})

fig.show()

# 3.2

In [ ]:
df_cnc3_prog_agg = df_cnc3.groupby(by=['c1_prog'], as_index=False) \
    .agg({'channel_cnt':'count','duration':'sum'}) \
    .rename(columns={'channel_cnt':'cnt'})

In [ ]:
df_cnc3_prog_agg.sort_values(by=['duration'], ascending=False)

In [ ]:
fig = px.bar(df_cnc3_prog_agg.sort_values(by=['duration'], ascending=False), x='c1_prog', y='duration', 
             title='Bar Chart of c1_prog with Duration',
             labels={'duration':'Duration'})

# Show the plot
fig.show()

# 3.3

In [ ]:
df_cnc3_mode_agg = df_cnc3.groupby(by=['c1_mode'], as_index=False) \
    .agg({'channel_cnt':'count','duration':'sum'}) \
    .rename(columns={'channel_cnt':'cnt'})

In [ ]:
df_cnc3_mode_agg['percentage'] = (df_cnc3_mode_agg['cnt'] / df_cnc3_mode_agg['cnt'].sum()) * 100

In [ ]:
fig = px.pie(df_cnc3_mode_agg, values='cnt', names='c1_mode', 
             title='Channel 1 mode percentage',
             hover_data=['percentage'], labels={'percentage':'Percentage'})

fig.show()

# 3.4

In [ ]:
# df_cnc3['date'] = df_cnc3['time_load'].dt.date
# df_cnc3['time'] = df_cnc3['time_load'].dt.floor('s').dt.time
# df_cnc3['hour'] = df_cnc3['time_load'].dt.hour

In [ ]:
df_cnc3['timestamp'] = df_cnc3['time_load'].dt.floor('s')

In [ ]:
df_cnc3.head()

In [ ]:
df_calc_online = df_cnc3.groupby(by=['timestamp'], as_index=False) \
    .agg({'channel_cnt':'count'}) \
    .rename(columns={'channel_cnt':'is_online'})

In [ ]:
df_calc_online['timestamp_msc'] = df_calc_online['timestamp'] + pd.Timedelta(hours=3)

In [ ]:
df_calc_online.set_index('timestamp_msc', inplace=True)

In [ ]:
df_resampled = df_calc_online.resample('S').asfreq(fill_value=0)

In [ ]:
df_resampled.reset_index(inplace=True)

In [ ]:
df_resampled['date'] = df_resampled['timestamp_msc'].dt.date

In [ ]:
df_resampled['duration'] = df_resampled['timestamp_msc'].diff()

In [ ]:
online_duration = df_resampled[df_resampled['is_online'] == 1]['duration'].sum()
offline_duration = df_resampled[df_resampled['is_online'] == 0]['duration'].sum()

In [ ]:
print(f'Total online duration: {online_duration}')
print(f'Total offline duration: {offline_duration}')

In [ ]:
online_duration_per_day = df_resampled[df_resampled['is_online'] == 1].groupby(df_resampled['timestamp_msc'].dt.date)['duration'].sum()
offline_duration_per_day = df_resampled[df_resampled['is_online'] == 0].groupby(df_resampled['timestamp_msc'].dt.date)['duration'].sum()

In [ ]:
print('Total online duration per date:')
print(online_duration_per_day)

print('\nTotal offline duration per date:')
print(offline_duration_per_day)

# 3.5

there were no errors in CNC 3 - thus there is nothing in dataframe

In [ ]:
# sql = """with error_time as	
# (select distinct c1_prog, date_trunc('second', time_load) time_load--* --distinct c1_stat
# from cnc_5_2
# where 1=1
# 	and c1_stat = 'Ошибка'
# order by time_load
# ), h as
# (select hump, date_trunc('second', time_load) time_load
# from sensor_hump_2
# ), t as
# (select temperature, date_trunc('second', time_load) time_load
# from sensor_temp_2
# )
# select *
# from error_time
# join h using(time_load)
# join t using(time_load)"""

In [ ]:
# df_h_t = postgresql_to_dataframe(conn, sql)

if run on local machine - then reading from postgres db can be used

instead - reading from csv file - but it is empty

In [ ]:
df_h_t = df_h_t.dropna()

In [ ]:
df_h_t['hump'] = df_h_t.hump.astype(int)

In [ ]:
df_h_t['temperature'] = df_h_t.temperature.astype(int)

In [ ]:
for p in df_h_t['c1_prog'].unique():
    print(f"Programm: {p}")
    display(df_h_t[df_h_t['c1_prog']==p].describe())

# 3.6

query to find when temperature and humidity were equal

there were no such for real mqtt

In [ ]:
# sql = """with h as
# (select hump, date_trunc('second', time_load) time_load
# from sens_h_2
# ), t as
# (select temperature, date_trunc('second', time_load) time_load
# from sens_t_2
# )
# select * 
# from h
# join t using(time_load)
# where 1=1
# 	and hump=temperature"""

In [ ]:
# df_sens_h_t = postgresql_to_dataframe(conn, sql)

In [ ]:
# df_sens_h_t

query to find when temperature and humidity were equal

In [ ]:
# sql =""" with h as
# (select hump, date_trunc('second', time_load) time_load
# from sensor_hump_2
# ), t as
# (select temperature, date_trunc('second', time_load) time_load
# from sensor_temp_2
# )
# select * 
# 	, abs(hump-temperature) as diff
# from h
# join t using(time_load)
# where 1=1
# 	and hump=temperature"""

In [ ]:
# df_sensor_h_t = postgresql_to_dataframe(conn, sql)

if run on local machine - then reading from postgres db can be used

instead - reading from csv file

In [ ]:
df_sensor_h_t = pd.read_csv('_with_h_as_select_hump_date_trunc_second_time_load_time_load_fro_202312191205.csv')

In [ ]:
df_sensor_h_t